In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [7]:
df_train_X = pd.read_csv('/home/clodis/Downloads/PlaneChalData/train_X.csv')
df_train_y = pd.read_csv('/home/clodis/Downloads/PlaneChalData/train_y.csv')
df_test_X = pd.read_csv('/home/clodis/Downloads/PlaneChalData/test1_X.csv')

In [3]:
columns_test = df_test_X.dropna(axis=1).columns
columns_train = df_train_X.dropna(axis=1).columns

In [4]:
for i in (columns_train == columns_test):
    if i != True:
        print(i)

In [5]:
columns_train.shape, columns_test.shape

((107,), (107,))

In [6]:
columns = columns_test

In [7]:
df_test_X = df_test_X[columns]
df_train_X = df_train_X[columns]
(df_train_X.isna().sum() != 0).sum()

np.int64(0)

In [8]:
(df_test_X.isna().sum() != 0).sum()

np.int64(0)

In [9]:
def preprocess_datetime(df):
    # Ensure columns are in datetime format
    df['report_date'] = pd.to_datetime(df['report_date'])
    df['contract_date'] = pd.to_datetime(df['contract_date'])

    # Extract year, month, day, weekday, etc.
    df['report_year'] = df['report_date'].dt.year
    df['report_month'] = df['report_date'].dt.month
    df['report_day'] = df['report_date'].dt.day
    df['report_weekday'] = df['report_date'].dt.weekday

    df['contract_year'] = df['contract_date'].dt.year
    df['contract_month'] = df['contract_date'].dt.month
    df['contract_day'] = df['contract_date'].dt.day
    df['contract_weekday'] = df['contract_date'].dt.weekday

    # Drop original datetime columns if no longer needed
    df = df.drop(columns=['report_date', 'contract_date'])
    
    # print([i for i in df.columns if 'date' in i])
    return df

In [10]:
X_train = preprocess_datetime(df_train_X)
X_test = preprocess_datetime(df_test_X)

In [11]:
X_train.head()

,contract_id,specialization_id,contract_init_sum,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,agg_cec_requests__g_contract__request_id__all__count__1W,agg_cec_requests__g_contract__request_id__all__count__2W,...,agg_materials__g_contract__material_type_id__countDistinct__ALL_TIME,agg_materials__g_contract__material_id__countDistinct__ALL_TIME,report_year,report_month,report_day,report_weekday,contract_year,contract_month,contract_day,contract_weekday
0,5433,18,1.115267,43,701,438,0.793952,-0.539030,-0.255766,0.341961,...,5.222766,6.111811,2023,1,1,6,2022,5,10,1
1,6875,18,1.608002,31,268,438,2.681675,0.743673,0.808046,0.662319,...,3.387956,5.004645,2023,1,1,6,2022,8,21,6
2,1476,18,-0.360764,31,268,438,-0.416432,-0.539030,-0.255766,-0.298754,...,-0.281663,0.308738,2023,1,1,6,2022,10,17,0
3,4469,12,-0.089303,43,697,484,-0.193827,-0.539030,-0.255766,0.021603,...,-0.159342,-0.187578,2023,1,1,6,2022,4,20,2
4,1330,12,-0.515778,49,224,484,0.133446,0.743673,-0.255766,-0.298754,...,-0.159342,-0.034865,2023,1,1,6,2022,3,2,2


In [12]:
X_test.head()

,contract_id,specialization_id,contract_init_sum,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,agg_cec_requests__g_contract__request_id__all__count__1W,agg_cec_requests__g_contract__request_id__all__count__2W,...,agg_materials__g_contract__material_type_id__countDistinct__ALL_TIME,agg_materials__g_contract__material_id__countDistinct__ALL_TIME,report_year,report_month,report_day,report_weekday,contract_year,contract_month,contract_day,contract_weekday
0,3029,15,-0.347029,42,84,637,-0.405169,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,24,0
1,4350,11,-0.092977,44,727,637,-0.196840,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,11,1
2,1095,15,-0.467957,44,728,637,-0.504333,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,24,0
3,2634,15,-0.486723,44,727,637,-0.519721,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,24,0
4,6535,11,-0.452585,42,85,637,-0.491727,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,27,3


In [13]:
df_train_y

,contract_id,report_date,default6
0,5433,2023-01-01,0
1,6875,2023-01-01,0
2,1476,2023-01-01,0
3,4469,2023-01-01,0
4,1330,2023-01-01,0
...,...,...,...
28826,5078,2023-07-23,0
28827,3854,2023-07-23,0
28828,5351,2023-07-23,0
28829,57,2023-07-23,0


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

train_X, test_X, train_y, test_y = train_test_split(X_train, df_train_y.default6, test_size=0.2, random_state=42)

# Define the XGBoost model
xgboost = XGBClassifier(use_label_encoder=True, eval_metric='aucpr')

# Define the parameter grid for GridSearchCV
parameters = {
    'n_estimators': [500],
    'learning_rate': [0.1],
    'max_depth': [7],
    'min_child_weight': [1],
    'gamma': [0],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

# Set up the GridSearchCV
clf = GridSearchCV(estimator=xgboost, param_grid=parameters, scoring='f1', cv=5, n_jobs=-1)

# Fit the model
clf.fit(train_X, train_y)

# Print the best parameters and best score
print("Best Parameters:", clf.best_params_)
print("Best CV Score:", clf.best_score_)

# Make predictions on the test set
y_pred = clf.predict(test_X)

# Evaluate the model
print("Test Set Accuracy:", accuracy_score(test_y, y_pred))
print("\nClassification Report:\n", classification_report(test_y, y_pred))

/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:46:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:46:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:46:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:46:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.8}
Best CV Score: 0.9715446551516024
Test Set Accuracy: 0.9932373851222472

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4848
           1       0.98      0.97      0.98       919

    accuracy                           0.99      5767
   macro avg       0.99      0.99      0.99      5767
weighted avg       0.99      0.99      0.99      5767



In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

train_X, test_X, train_y, test_y = train_test_split(X_train, df_train_y.default6, test_size=0.2, random_state=42)

boost = XGBClassifier(eval_metric='aucpr', n_estimators = 500)
boost.fit(X_train, df_train_y.default6)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [16]:
from sklearn .metrics import roc_auc_score
auc = roc_auc_score(test_y, y_pred)
auc * 2 - 1

np.float64(0.9716724664849512)

In [17]:
ans = pd.read_csv('submission.csv')
ans.score = clf.predict(X_test)

In [18]:
ans.score.value_counts()

score
0    6888
1     444
Name: count, dtype: int64

In [40]:
# Всё ок порядок сохранён
ans.score = 1 - clf.predict_proba(X_test)
ans.score.unique()

array([0.00235605, 0.01144212, 0.03982949, ..., 0.10646254, 0.00190502,
       0.01367205], dtype=float32)

In [41]:
ans.to_csv('submission_model_1.csv', index = False)

In [21]:
ans

,contract_id,report_date,score
0,3029,2023-07-30,0.002356
1,4350,2023-07-30,0.011442
2,1095,2023-07-30,0.039829
3,2634,2023-07-30,0.009016
4,6535,2023-07-30,0.000732
...,...,...,...
7327,4740,2023-09-24,0.001761
7328,2689,2023-09-24,0.052340
7329,7135,2023-09-24,0.106463
7330,4164,2023-09-24,0.001905


In [22]:
df_test_X.report_date
ans.report_date
df_test_X.contract_id
ans.contract_id

0       3029
1       4350
2       1095
3       2634
4       6535
        ... 
7327    4740
7328    2689
7329    7135
7330    4164
7331     194
Name: contract_id, Length: 7332, dtype: int64

In [23]:
ans.max()

contract_id          7338
report_date    2023-09-24
score             0.99069
dtype: object

In [24]:
((X_test.columns == X_train.columns) != True).sum()

np.int64(0)

In [43]:
# TEST
ans1 = ans.copy()
ans1.score = 1 - boost.predict_proba(X_test)

In [44]:
ans1.score.value_counts()

score
7.748604e-07    12
9.536743e-07     9
1.132488e-06     8
4.827976e-06     7
7.152557e-07     7
                ..
1.450777e-04     1
2.603037e-01     1
4.816830e-03     1
3.334165e-03     1
1.027763e-02     1
Name: count, Length: 6510, dtype: int64

In [45]:
ans1.head()

,contract_id,report_date,score
0,3029,2023-07-30,0.003334
1,4350,2023-07-30,0.004817
2,1095,2023-07-30,0.260304
3,2634,2023-07-30,0.020526
4,6535,2023-07-30,0.000146


In [46]:
ans1.to_csv('submission_model_2.csv', index = False)

In [29]:
# TEST
ans1 = ans.copy()
ans1.score = boost.predict(X_test)
ans1.score.value_counts()

score
0    6774
1     558
Name: count, dtype: int64

In [33]:
# Combine models predictions
ans.head()

,contract_id,report_date,score
0,3029,2023-07-30,0.002356
1,4350,2023-07-30,0.011442
2,1095,2023-07-30,0.039829
3,2634,2023-07-30,0.009016
4,6535,2023-07-30,0.000732


In [34]:
ans1.head()

,contract_id,report_date,score
0,3029,2023-07-30,0.003334
1,4350,2023-07-30,0.004817
2,1095,2023-07-30,0.260304
3,2634,2023-07-30,0.020526
4,6535,2023-07-30,0.000146


In [61]:
ans2 = ans.copy()
ans2.score = (ans.score + ans1.score * 2) / 3
ans2

,contract_id,report_date,score
0,3029,2023-07-30,0.003008
1,4350,2023-07-30,0.007025
2,1095,2023-07-30,0.186812
3,2634,2023-07-30,0.016690
4,6535,2023-07-30,0.000341
...,...,...,...
7327,4740,2023-09-24,0.000910
7328,2689,2023-09-24,0.026787
7329,7135,2023-09-24,0.043009
7330,4164,2023-09-24,0.000732


In [65]:
ans2.head()

,contract_id,report_date,score
0,3029,2023-07-30,0.003008
1,4350,2023-07-30,0.007025
2,1095,2023-07-30,0.186812
3,2634,2023-07-30,0.016690
4,6535,2023-07-30,0.000341


In [66]:
ans2.to_csv('Model_combine.csv', index = False)

In [64]:
ans2.score.max()

np.float32(0.9941876)